## PAIR PROGRAMMING ETL III

### ETL Transformación II - Clases y Funciones de limpieza
---

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import ast

In [2]:
class Ataquesclima:

    def __init__(self, dicc_datos):

        self.dicc_datos= dicc_datos

    def conexion_API(self, producto):
        
        # Especificamos el producto del que queremos extraer la información
        self.producto= producto
       
        # Creamos un dataframe vacío con las columnas que rellenaremos con la información obtenida de la API
        df_clima_paises = pd.DataFrame( columns= ['timepoint', 'cloudcover', 'lifted_index', 'prec_type', 'prec_amount','temp2m', 'rh2m', 'weather', 'wind10m.direction', 'wind10m.speed','country'])

        for key, value in self.dicc_datos.items(): 

            url =f'http://www.7timer.info/bin/api.pl?lon=-{self.dicc_datos[key][0]}&lat={self.dicc_datos[key][1]}&product={self.producto}&output=json'

            response = requests.get(url=url)
            response.status_code
            response.reason
            if codigo_estado == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 402:
                print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 404:
                print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            else:
                print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

            # Convetimos la información obtenida en formato json a un dataframe
            df = pd.json_normalize(response.json()['dataseries']) 
            # Creamos una nueva columna para que inserte la key que corresponde al nombre del país
            df['country']=key
            # Concatenamos los dataframes de cada país en uno
            df_clima_paises=pd.concat([df_clima_paises,df], axis=0, ignore_index = True)
            return df_clima_paises


    def limpiar_meteo(self, df_clima_paises): #(((((pendiente de cambiar a df, ya que puede ir cualquier df))))

        df_clima_paises['rh_profile']= df_clima_paises['rh_profile'].apply(ast.literal_eval)
        # Para separar la lista de diccionarios en varias columnas
        x = df_clima_paises['rh_profile'].apply(pd.Series)     

        # For loop para sacar el nombre de la columna y los valores de las filas
        for i in range(len(x.columns)): 
    
            # aplicamos el apply,extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(x[i].apply(pd.Series)["rh"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_clima_paises.insert(i, nombre, valores)

            

        df_clima_paises['wind_profile']= df_clima_paises['wind_profile'].apply(ast.literal_eval)
        y = df_clima_paises['wind_profile'].apply(pd.Series)

        # For loop para sacar el nombre de la columna y los valores de las filas
        for i in range(len(y.columns)): 
            
            # aplicamos el apply,extraemos los valores de la key "layer" y lo almacenamos en dos variables que convertimos a strings
            nombre = "direction" + str(y[i].apply(pd.Series)["layer"][0]) 
            nombre2 = "speed" + str(y[i].apply(pd.Series)["layer"][0]) 

            # hacemos lo mismo con dos variables para "guardar" los valores
            valores = list(y[i].apply(pd.Series)["direction"] )
            valores2= list(y[i].apply(pd.Series)["speed"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_clima_paises.insert(i, nombre, valores)
            df_clima_paises.insert(i,nombre2,valores2)



            # Eliminamos las columnas que tienen las listas de diccionarios, información duplicada
            df_clima_paises.drop(['rh_profile','wind_profile'], axis=1, inplace=True)

            #Calculamos la media por pais
            df_clima_paises = df_clima_paises.groupby('country').mean()

            #Calculamos la fecha del dato y añadimos la columna al dataframe
            hoy = datetime.now()
            hoy = datetime.strftime(hoy, '%Y-%m-%d')
            df_clima_paises["fecha"] = hoy

            #reseteamos el indice para guardar correctamente el df.
            df_clima_paises.reset_index(inplace=True)

            return df_clima_paises

    def juntar_dfs(self, df_attacks, df_clima_paises): 

        df_union= df_attacks.merge(df_clima_paises, how= 'inner', on= 'country')


        print("El df de union es ", df_union.columns)
        print("-----------------------------------------")

                
        # guardamos los datos
        df_union.to_pickle('../files/datos_actualizados.pkl')
        df_union.to_csv('../files/datos_actualizados.csv')

        return df_union

    def chequear_datos(self, df_union): 
    
        print("Las columnas son:", "\n")
        print(list(df_union.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df_union.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df_union.isnull().sum() / df_union.shape[0]) *  100)



            

In [3]:
 # Hacemos un diccionario con los países como keys y las coordenadas como values
dicc_datos= {'usa': [-100.445882, 39.7837304],'australia': [134.755, -24.7761086],'south africa': [24.991639, -28.8166236],'new zealand': [172.8344077, -41.5000831],'papua new guinea': [144.2489081, -5.6816069]}

In [4]:
api = Ataquesclima(dicc_datos) # Iniciamos la clase

In [5]:
df= api.conexion_API('meteo')

NameError: name 'codigo_estado' is not defined